In [ ]:
    import numpy as np
    from astropy.table import Table
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    import numpy as np
    import csv
    import seaborn as sns
    import matplotlib.pyplot as plt
    import statsmodels.api as sm
    from scipy.stats import pearsonr
    from kneed import KneeLocator
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    from astropy.io import fits



    # Read the data from the FITS file
    filename = 'C:/Users/pedro/Downloads/StellarMasses.fits'
    table = Table.read(filename)

    n_datapoints = len(table)
    print(f'The dataset contains {n_datapoints} datapoints.')

    # Convert the table to a pandas dataframe
    df = table.to_pandas()

    #drop redundant features
    redundant_cols = df.filter(like='absmag_').columns.difference(df.filter(like='absmag_*_stars').columns)
    df.drop(redundant_cols, axis=1, inplace=True)

    # Select only the numerical columns for normalization
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_data = df[numeric_cols]

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(numeric_data)
    
    normalized_df = pd.DataFrame(normalized_data, columns=numeric_cols)

    # Calculate the correlation coefficient between each feature and the redshift
    correlations = normalized_df.corr()['Z']


    # Select the top features with the highest absolute correlation coefficient
    n_features = 100  # Select the number of top features to display
    top_features = abs(correlations).nlargest(n_features).index.tolist()

    # Print the names of the top features
    print(f'Top {n_features} features that correlate with redshift:')
    for feature in top_features:
        print(feature)



    # Select the features for clustering
    features = ['S2N', 'dellogLWage', 'dellogmoverl_i', 'nQ', 'PPP','Z','logmstar']
    
    



    # Create a new dataframe with only the selected features
    X = normalized_df[features]
    #Separate target variable from other features
    X = X.drop('Z', axis=1)
    Y=normalized_df[features]['Z']
    print(f"This is Z {Y}" )

    # Set the number of clusters
    n_clusters = 3

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X)


    normalized_df['cluster'] = kmeans.labels_


    # determine the optimal value of k
    kl = KneeLocator(K, distortions, curve="convex", direction="decreasing")
    optimal_k = kl.elbow
    print("Optimal k value: ", optimal_k)




    # Apply K-means clustering to select the most important features
    kmeans = KMeans(n_clusters=5, random_state=42)
    X_clusters = kmeans.fit_predict(X)
    important_features = []
    for cluster in range(5):
        cluster_indices = X_clusters == cluster
        cluster_X = X.loc[cluster_indices, :]
        cluster_kmeans = KMeans(n_clusters=3, random_state=42)
        cluster_kmeans.fit(cluster_X)
        cluster_center = cluster_kmeans.cluster_centers_.mean(axis=0)
        important_feature = cluster_X.columns[abs(cluster_X.mean() - cluster_center).argmax()]
        important_features.append(important_feature)

    # Select the important features and split the dataset into training and testing sets
    X = X[important_features]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Train a linear regression model on the selected features
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # Evaluate the model on the testing set
    score = lr.score(X_test, y_test)
    print(f"Model accuracy: {score}")


